In [136]:
KAGGLE = False #if getpass.getuser() == "Kaihua" else True

import os, copy, zipfile, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from functools import reduce
from sklearn.metrics import mean_squared_error

In [137]:
import warnings
warnings.filterwarnings("ignore")

In [138]:
#!pip3 install lightgbm

In [139]:
import sys

DATA_PATH = ''


SEED = 2024
cat_feats = []

In [140]:
train_data = pd.read_parquet(os.path.join(DATA_PATH, 'track3_train.parquet'))
test_data = pd.read_parquet(os.path.join(DATA_PATH, 'track3_a.parquet'))

train_data['type'] = 'train'
test_data['type'] = 'test'
data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)
data = pd.concat([data, data['ID'].str.split('_', expand=True).rename(columns={0:'station',1:'sample',2:'time'})], axis=1)#细化向关联项

print(f'train_data: {train_data.shape}')
print(f'test_data: {test_data.shape}')
print(f'all: {data.shape}')


train_data: (791616, 77)
test_data: (216384, 74)
all: (1008000, 80)


In [141]:
TARGET_FEATS = ['wdir_2min', 'spd_2min', 'spd_inst_max']

In [142]:
# 删除有问题的目标
for item in TARGET_FEATS:
    idx = data[item] >= 199999.0
    data.loc[idx, item] = np.nan

In [143]:
data['station'] = data['station'].apply(lambda x: int(x.split('D')[-1]))
cat_feats += ['station']

data['sample'] = data['sample'].apply(lambda x: int(x.split('Sample')[-1]))
data['time'] = data['time'].astype(int)

In [144]:
feats = [item for item in data.columns if item not in TARGET_FEATS+['ID', 'sample', 'type']]#剔除不相关项
print(f'feats: {len(feats)}, {feats}')
print(cat_feats)

feats: 74, ['100u', '100v', '10u', '10v', '2d', '2t', 'cape', 'capes', 'cp', 'deg0l', 'lcc', 'msl', 'skt', 'sp', 'sst', 'tcc', 'd_L1000', 'q_L1000', 'r_L1000', 't_L1000', 'u_L1000', 'v_L1000', 'w_L1000', 'd_L950', 'q_L950', 'r_L950', 't_L950', 'u_L950', 'v_L950', 'w_L950', 'd_L925', 'q_L925', 'r_L925', 't_L925', 'u_L925', 'v_L925', 'w_L925', 'd_L900', 'q_L900', 'r_L900', 't_L900', 'u_L900', 'v_L900', 'w_L900', 'd_L850', 'q_L850', 'r_L850', 't_L850', 'u_L850', 'v_L850', 'w_L850', 'd_L700', 'q_L700', 'r_L700', 't_L700', 'u_L700', 'v_L700', 'w_L700', 'd_L500', 'q_L500', 'r_L500', 't_L500', 'u_L500', 'v_L500', 'w_L500', 'd_L200', 'q_L200', 'r_L200', 't_L200', 'u_L200', 'v_L200', 'w_L200', 'station', 'time']
['station']


In [145]:
data

,ID,wdir_2min,spd_2min,spd_inst_max,100u,100v,10u,10v,2d,2t,...,q_L200,r_L200,t_L200,u_L200,v_L200,w_L200,type,station,sample,time
0,D0011_Sample00000_01,88.0,4.5,5.2,-3.178867,1.520631,-3.035558,0.921220,-4.296475,2.166330,...,0.005066,7.282292,-54.574743,60.027509,4.458829,0.052047,train,1101,0,1
1,D0011_Sample00000_02,101.0,4.5,6.0,-3.044837,1.737420,-2.924957,1.208521,-3.802847,2.137752,...,0.005006,6.950891,-54.490239,58.961815,3.799950,0.053916,train,1102,0,2
2,D0011_Sample00000_03,111.0,3.9,5.1,-2.910807,1.954209,-2.814357,1.495822,-3.309219,2.109175,...,0.004946,6.619490,-54.405734,57.896121,3.141070,0.055785,train,1103,0,3
3,D0011_Sample00000_04,160.0,1.7,4.3,-1.870547,1.442103,-1.843161,1.229792,-3.500122,1.850405,...,0.004919,6.931894,-54.718634,57.157805,2.129127,0.064269,train,1104,0,4
4,D0011_Sample00000_05,179.0,1.3,2.9,-0.830286,0.929998,-0.871964,0.963762,-3.691025,1.591636,...,0.004893,7.244298,-55.031533,56.419490,1.117184,0.072752,train,1105,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007995,D0009_Sample02128_44,NaN,NaN,NaN,3.843052,8.813500,3.456899,8.086399,-6.134135,4.715720,...,0.005002,7.706795,-54.882592,58.070532,-1.666188,0.017625,test,944,2128,44
1007996,D0009_Sample02128_45,NaN,NaN,NaN,3.674979,9.288052,3.270604,8.504519,-6.362700,5.081217,...,0.005217,8.115371,-54.979291,58.097675,-2.525959,-0.009414,test,945,2128,45
1007997,D0009_Sample02128_46,NaN,NaN,NaN,4.163034,9.299423,3.684067,8.492985,-5.268479,5.118454,...,0.005117,7.934781,-54.952597,58.558392,-3.415087,0.020155,test,946,2128,46
1007998,D0009_Sample02128_47,NaN,NaN,NaN,4.651088,9.310794,4.097529,8.481452,-4.174258,5.155692,...,0.005016,7.754191,-54.925903,59.019110,-4.304215,0.049725,test,947,2128,47


In [ ]:
train_data = data.query('type=="train"').reset_index(drop=True)
test_data = data.query('type=="test"').reset_index(drop=True)

In [ ]:
lgb_params = {  # 用于控制模型训练的超参数字典，如学习率、树的深度、子采样比例等
    'boosting_type': 'gbdt', # 默认参数，基于梯度提升决策树的方法，可以用于回归和分类问题，速度较快、准确度较高
    'objective': 'quantile', #mse mape
    'metric': 'quantile',
    # 'max_depth': 6,
    'num_leaves': 2 ** 6, # 默认31  越大模型复杂度越高易过拟合，反之欠拟合
    # 'num_leaves': 31,
    'min_data_in_leaf': 20,  # TODO 参数调小，这样可以提高模型的复杂度，提高模型的拟合能力
#     'lambda_l1': 0.5,  
#     'lambda_l2': 0.5,  
    'feature_fraction': 0.7,   #每一次树木生长时选择多少特征。常用的取值范围是0.5到1
    'bagging_fraction': 0.7,  #控制在每次迭代中，模型将使用多少数据。0.5到1是一个常见的取值范围
    'bagging_freq': 5,  # 每n次迭代进行子采样，减少子采样的频率可以提高准确性，但会增加训练时间。默认为0
    'learning_rate': 0.01,  #取值范围一般是0.001到1之间 较小的学习率会导致模型需要更多的 迭代次数 才能收敛，但更大的学习率可能会导致模型不收敛或过拟合。一般调小
    'n_jobs': -1,
    'verbose': -1,
    "device_type": "cpu",
    'feature_fraction_seed':SEED, #控制特征选择过程中的随机种子，用于生成不同的特征
    'bagging_seed':SEED, #控制bagging操作的随机种子，用于生成不同的训练样本
    'seed': SEED, #控制数据的随机排序和划分。默认为0，即每次运行时都会产生相同的结果
}


In [ ]:
# 逐小时预报采用四舍五入保留1位小数位
# 24小时内最大风由保留1位小数位的逐小时平均风速求最大值
# 24小时内极大风由保留1位小数位的逐小时极大风速求最大值


In [ ]:
task_name = "lgb"
task_params = {"lgb": lgb_params}[task_name] # 用于控制模型训练的超参数字典，如学习率、树的深度、子采样比例等

In [ ]:
# train_oof = {}
test_pred = {}
feats_importance = {}

for item_target in TARGET_FEATS:
    train_y = train_data[item_target]
    idx = ~train_y.isna()

    # trian_id = train_data.loc[idx, 'ID'].reset_index(drop=True)  #原始训练集中ID列+为缺失值的结果列
    train_x = train_data.loc[idx, feats].reset_index(drop=True)  #训练集（已细化并剔除不相关）+为缺失值的结果列
    testA_x = test_data[feats].reset_index(drop=True) #测试集（已细化并剔除不相关）
    train_y = train_y.loc[idx].reset_index(drop=True) #缺失值的结果列
    group_x = train_data.loc[idx, 'sample'].reset_index(drop=True) #原始训练集中sample列+为缺失值的结果列
    print(train_x.shape, testA_x.shape)

    # item_oof = np.zeros(train_x.shape[0])
    item_pred = np.zeros(testA_x.shape[0])

    fold_num = 5
    item_importance = 0
    from sklearn.model_selection import GroupKFold #用于分组数据的交叉验证迭代器 测试模型的泛化能力 检测过拟合情况
    import lightgbm as lgb #轻量级梯度提升  基于学习算法的决策树  分布式、高效
    kf = GroupKFold(n_splits=fold_num, ) #n_splits一般3-10之间  数据集越大n_splits调小  越大越精准但泛化能力下降过拟合增大   需要找平衡点
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_x, groups=group_x)): #基于GroupKFold交叉验证分组，根据sample标识（文件夹）分组  保证每个分组样本均匀分配到不同训练集和验证集中
        print('-----------', fold)
        train = lgb.Dataset(   # 数据集封装成一个对象、加载入内存 方便LightGBM 进行训练和预测
            train_x.loc[train_idx], #训练集
            train_y.loc[train_idx],
            categorical_feature=cat_feats # 类别特征列表，用于指定数据集中哪些特征是类别型的特征
        )
        val = lgb.Dataset(
            train_x.loc[val_idx], #验证集
            train_y.loc[val_idx],
            categorical_feature=cat_feats # cat_feats=['station']
        )
        model = lgb.train(task_params, train, valid_sets=[train, val], num_boost_round=90_000,
                            callbacks=[lgb.early_stopping(100), lgb.log_evaluation(100)])
        #valid_sets 验证数据集为什么还要加上train
        """
        params: 用于控制模型训练的超参数字典，如学习率、树的深度、子采样比例等。
        train_set: 训练数据集，可以是 lgb.Dataset 对象或数据矩阵。
        num_boost_round: 迭代次数，即弱分类器的数量。
        valid_sets: 验证数据集，可以是一个 lgb.Dataset 对象或一个列表，其中每个元素都是 lgb.Dataset 对象。
        early_stopping_rounds: 早停次数，如果在连续的 early_stopping_rounds 次迭代中验证集的损失没有得到改善，则停止迭代。
            lgb.early_stopping()比这个好用，可以把num_boost_round放的很大，他可以智能判断早停
        callbacks: 回调函数列表，用于在训练过程中执行特定操作，如打印日志、保存模型等。
        """
        # item_oof[val_idx] += (model.predict(train_x.loc[val_idx])) #预测验证集结果存入item_oof

        item_pred += (model.predict(testA_x))/fold_num  #预测测试集/折数=平局值 item_pred
        item_importance += model.feature_importance(importance_type='gain') / fold_num # 特征性/折数=平均值 importance_type=split/gain 分别表示特征被用于分裂的次数和特征对模型的贡献度

    importance = pd.DataFrame()
    importance['name'] = feats
    importance['importance'] = item_importance

    # train_oof[item_target] = pd.DataFrame({"ID": trian_id, f"{item_target}_true": train_y, f"{item_target}_pred": item_oof}) #重构预测的验证集结果列
    test_pred[item_target] = pd.DataFrame({"ID": test_data['ID'], f"{item_target}": item_pred})
    feats_importance[item_target] = importance #重构预测的测试集结果列

In [ ]:
# train_oof = reduce(lambda x,y: pd.merge(x,y,on='ID', how='outer'), train_oof.values())
test_pred = reduce(lambda x,y: pd.merge(x,y,on='ID', how='outer'), test_pred.values())
# train_oof.shape
test_pred.shape

score_str = 'potato_90_stationTime'

In [ ]:
if not os.path.exists("result"):
    os.makedirs("result")
# train_oof.to_csv(os.path.join("result", f'lgb_oof_{score_str}.csv'), index=False)
test_pred.to_csv(os.path.join("result", f'{score_str}.csv'), index=False)